In [1]:
# 필요한 라이브러리 설치
!pip install transformers torch
!pip install --upgrade gluonnlp pandas tqdm
!pip install accelerate -U

# 라이브러리 임포트
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler, SequentialSampler, random_split
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
from transformers import ElectraModel, ElectraTokenizer
import numpy as np
from tqdm.auto import tqdm
from google.colab import drive
import glob
import os
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, mean_squared_error

# Google Drive 마운트
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 117.3 MB/s eta 0:00:00
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661659 sha256=caee3433dfa9acbdbd35bccf31563c08be3b4ad09605256e73f410da00f9879f
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.

#훈련 및 평가

In [ ]:
import os
import torch
import pandas as pd
from transformers import ElectraTokenizer, ElectraModel, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, SequentialSampler
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import torch.nn as nn

# KoElectra 토크나이저와 모델 불러오기
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

# 회귀를 위한 모델 정의
class ElectraForCompanyRegression(nn.Module):
    def __init__(self, model_name):
        super(ElectraForCompanyRegression, self).__init__()
        self.electra = ElectraModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(self.electra.config.hidden_size + 768, 1)

    def forward(self, input_ids, attention_mask, company_embeddings):
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        combined_output = torch.cat((pooled_output, company_embeddings), dim=1)
        combined_output = self.dropout(combined_output)
        logits = self.regressor(combined_output)
        return logits

# 기업명 임베딩 생성
def get_company_embedding(company_name, tokenizer, model):
    inputs = tokenizer(company_name, return_tensors="pt").to('cuda')  # Ensure input is on GPU
    with torch.no_grad():
        outputs = model.electra(**inputs)
    company_embedding = outputs.last_hidden_state[:, 0, :]
    return company_embedding

def preprocess_with_company(data, tokenizer, model, max_len=256):
    texts = data["full_text"].astype(str).tolist()
    labels = data["evaluation_1"].fillna(0).astype(float).tolist()
    company_names = data["기업명"].astype(str).tolist()

    inputs = tokenizer(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to('cuda')  # Ensure inputs are on GPU

    labels = torch.tensor(labels, dtype=torch.float).to('cuda')  # Move labels to GPU

    # 기업명 임베딩 생성 및 병합
    company_embeddings = []
    for company_name in tqdm(company_names, desc="Generating company embeddings"):
        embedding = get_company_embedding(company_name, tokenizer, model).to('cuda')
        company_embeddings.append(embedding)
    company_embeddings = torch.cat(company_embeddings, dim=0)

    return inputs, labels, company_embeddings

# 데이터셋 로드
file_path = '/content/drive/MyDrive/Kwargs/적합성/anonymized_data.csv'
data = pd.read_csv(file_path)

# 모델 초기화
model_name = "monologg/koelectra-base-discriminator"
model = ElectraForCompanyRegression(model_name).to('cuda')  # Ensure model is on GPU

# 데이터 전처리
print("Preprocessing data...")
inputs, labels, company_embeddings = preprocess_with_company(data, tokenizer, model)

# TensorDataset 생성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], company_embeddings, labels)

# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더 생성
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=16)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)

# 모든 텐서를 GPU로 이동
def to_device(batch, device):
    return [x.to(device) for x in batch]

# 훈련 함수
def train(model, train_dataloader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc="Training"):
        batch = to_device(batch, device)
        input_ids, attention_mask, company_embeddings, labels = batch
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask, company_embeddings).squeeze()
        loss = nn.MSELoss()(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_dataloader)

# 평가 함수
def evaluate(model, val_dataloader, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc="Evaluating"):
            batch = to_device(batch, device)
            input_ids, attention_mask, company_embeddings, labels = batch
            logits = model(input_ids, attention_mask, company_embeddings).squeeze()
            loss = nn.MSELoss()(logits, labels)
            total_loss += loss.item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(logits.cpu().numpy())
    mse = mean_squared_error(all_labels, all_preds)
    return total_loss / len(val_dataloader), mse

# 훈련 및 평가 루프
num_epochs = 3
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    train_loss = train(model, train_dataloader, optimizer, 'cuda')
    val_loss, val_mse = evaluate(model, val_dataloader, 'cuda')
    print(f"Training Loss: {train_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation MSE: {val_mse:.4f}")

    # 모델 저장
    epoch_output_dir = f"/content/drive/MyDrive/Kwargs/적합성/모델_epoch_{epoch + 1}"
    if not os.path.exists(epoch_output_dir):
        os.makedirs(epoch_output_dir)

    # 모델 가중치 저장
    torch.save(model.state_dict(), os.path.join(epoch_output_dir, "model.pt"))

    # 토크나이저 저장
    tokenizer.save_pretrained(epoch_output_dir)
    print(f"Model and tokenizer saved to {epoch_output_dir} at Epoch {epoch + 1}")


Preprocessing data...


Generating company embeddings: 100%|██████████| 85771/85771 [14:00<00:00, 102.00it/s]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Evaluating: 100%|██████████| 1073/1073 [00:57<00:00, 18.56it/s]


Training Loss: 0.0868
Validation Loss: 0.0785
Validation MSE: 0.0785
Model and tokenizer saved to /content/drive/MyDrive/Kwargs/적합성/모델_epoch_1 at Epoch 1
Epoch 2/3


Evaluating: 100%|██████████| 1073/1073 [00:57<00:00, 18.55it/s]


Training Loss: 0.1168
Validation Loss: 0.1231
Validation MSE: 0.1231
Model and tokenizer saved to /content/drive/MyDrive/Kwargs/적합성/모델_epoch_2 at Epoch 2
Epoch 3/3


Evaluating:  94%|█████████▍| 1006/1073 [00:54<00:03, 18.51it/s]

# 완성된 모델로 로우데이터 평가

In [2]:
import torch
import pandas as pd
from transformers import ElectraTokenizer
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
from tqdm.auto import tqdm

# 모델 정의 클래스 가져오기 (이미 위에 정의된 것으로 가정)
class ElectraForCompanyRegression(nn.Module):
    def __init__(self, model_name):
        super(ElectraForCompanyRegression, self).__init__()
        self.electra = ElectraModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.regressor = nn.Linear(self.electra.config.hidden_size + 768, 1)

    def forward(self, input_ids, attention_mask, company_embeddings):
        outputs = self.electra(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # [batch_size, hidden_size]
        combined_output = torch.cat((pooled_output, company_embeddings), dim=1)
        combined_output = self.dropout(combined_output)
        logits = self.regressor(combined_output)
        return logits

# 모델과 토크나이저 로드
model_path = '/content/drive/MyDrive/Kwargs/적합성/모델_epoch_1'
model_name = "monologg/koelectra-base-discriminator"
model = ElectraForCompanyRegression(model_name)
model.load_state_dict(torch.load(f"{model_path}/model.pt"))
model.to('cuda')
model.eval()

tokenizer = ElectraTokenizer.from_pretrained(model_path)

# 기업명 임베딩 생성
def get_company_embedding(company_name, tokenizer, model):
    inputs = tokenizer(company_name, return_tensors="pt").to('cuda')  # Ensure input is on GPU
    with torch.no_grad():
        outputs = model.electra(**inputs)
    company_embedding = outputs.last_hidden_state[:, 0, :]
    return company_embedding

# 새로운 데이터 전처리
def preprocess_with_company(data, tokenizer, model, max_len=256):
    texts = data["full_text"].astype(str).tolist()
    company_names = data["기업명"].astype(str).tolist()

    inputs = tokenizer(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to('cuda')  # Ensure inputs are on GPU

    # 기업명 임베딩 생성 및 병합
    company_embeddings = []
    for company_name in tqdm(company_names, desc="Generating company embeddings"):
        embedding = get_company_embedding(company_name, tokenizer, model).to('cuda')
        company_embeddings.append(embedding)
    company_embeddings = torch.cat(company_embeddings, dim=0)

    return inputs, company_embeddings

# 새로운 CSV 데이터 로드
file_path = '/content/drive/MyDrive/Kwargs/뉴스 크롤링/csv/LG에너지솔루션_news_data.csv'  # 예측할 CSV 파일 경로
data = pd.read_csv(file_path)

# 데이터 전처리
print("Preprocessing data...")
inputs, company_embeddings = preprocess_with_company(data, tokenizer, model)

# TensorDataset 생성
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], company_embeddings)

# 데이터 로더 생성
dataloader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=16)

# 예측 함수
def predict(model, dataloader, device):
    model.eval()
    all_preds = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Predicting"):
            batch = [x.to(device) for x in batch]
            input_ids, attention_mask, company_embeddings = batch
            logits = model(input_ids, attention_mask, company_embeddings).squeeze()
            all_preds.extend(logits.cpu().numpy())
    return all_preds

# 모델을 사용하여 예측 수행
print("Predicting labels for new data...")
predictions = predict(model, dataloader, 'cuda')

# 예측 결과를 원래 데이터와 함께 저장
data["evaluation_1"] = predictions
output_path = '/content/drive/MyDrive/Kwargs/적합성/company_related_LG에너지솔루션.csv'
data.to_csv(output_path, index=False)

print(f"Predicted results saved to {output_path}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

<ipython-input-2-b1f562a8c68c>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{model_path}/model.pt"))


Preprocessing data...


KeyError: 'full_text'